# TP03 — Inverse Kinematics (Session 1)

**GEII3 — Industrie 4.0: Robotique | Lab Session 3.1**

**Duration**: 4 hours  
**Focus**: 2R Planar Robot - Analytical IK

---

## Learning Objectives (Session 1)

1. Understand the **inverse kinematics problem** for simple robots
2. Derive and implement **analytical/geometric IK** for 2R planar
3. Handle **multiple solutions** (elbow up/down)
4. Work with both **DH and ETS** representations (modern approach)
5. Validate with **Robotics Toolbox**
6. Analyze **workspace boundaries** and **reachability**

---

## The Inverse Kinematics Problem

### Problem Statement

**Given**: Desired end-effector pose $T_d \in SE(3)$ (position + orientation)

**Find**: Joint configuration $\mathbf{q} = [q_1, q_2, ..., q_n]^T$ such that:

$$T(\mathbf{q}) = T_d$$

where $T(\mathbf{q})$ is the forward kinematics function.

For **2R planar robot** (2-DOF), we only control position $(x, y)$:

$$\begin{aligned}
x &= L_1\cos(q_1) + L_2\cos(q_1+q_2) \\
y &= L_1\sin(q_1) + L_2\sin(q_1+q_2)
\end{aligned}$$

### Key Challenges

1. **Existence**: Target must be within workspace
2. **Uniqueness**: Two solutions possible (elbow up/down)
3. **Singularities**: At full extension or full retraction

---

## Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt, acos

# RTB (if available)
try:
    import roboticstoolbox as rtb
    from spatialmath import SE3
    RTB_AVAILABLE = True
    print(f"✓ RTB version: {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False
    print("⚠ RTB not available (optional)")

# Import helper functions
from tp_02_fk_helpers import (
    fk_dh, fk_ets, rotx, roty, rotz, transx, transy, transz,
    extract_position, extract_rotation
)

from tp_03_ik_helpers import (
    plot_robot_2d, compare_solutions, normalize_angle,
    create_rtb_robot_from_dh, create_rtb_robot_from_ets
)

np.set_printoptions(precision=4, suppress=True)
print("✓ Setup complete")

---

# Exercise 1: 2R Planar - Analytical IK

**Time**: 80 minutes

## Robot Description

- Link 1: $L_1 = 0.4$ m
- Link 2: $L_2 = 0.3$ m
- Workspace: Annulus with $r \in [|L_1-L_2|, L_1+L_2] = [0.1, 0.7]$ m

## Task 1.1: Geometric Solution Derivation (20 min)

### Law of Cosines Approach

Given target $(x_d, y_d)$:

**Step 1**: Distance to target
$$D = \sqrt{x_d^2 + y_d^2}$$

**Step 2**: Reachability check
$$|L_1 - L_2| \leq D \leq L_1 + L_2$$

**Step 3**: Solve for $q_2$ (law of cosines)
$$\cos(q_2) = \frac{D^2 - L_1^2 - L_2^2}{2 L_1 L_2}$$

Two solutions:
- Elbow UP: $q_2 = +\arccos(\cos q_2)$
- Elbow DOWN: $q_2 = -\arccos(\cos q_2)$

**Step 4**: Solve for $q_1$
$$q_1 = \text{atan2}(y_d, x_d) - \text{atan2}(L_2\sin q_2, L_1 + L_2\cos q_2)$$

---

## Task 1.2: Implementation (30 min)

In [ ]:
# Robot parameters
L1 = 0.4
L2 = 0.3

print(f"2R Planar Robot:")
print(f"  L1 = {L1} m")
print(f"  L2 = {L2} m")
print(f"  Workspace: r ∈ [{abs(L1-L2):.1f}, {L1+L2:.1f}] m")

# DH representation (Standard convention)
dh_2r = [
    (0, 0, L1, 0, 'r'),  # Joint 1: Rz(q1) + Tx(L1)
    (0, 0, L2, 0, 'r'),  # Joint 2: Rz(q2) + Tx(L2)
]

# ETS representation (Modern approach from TP02)
ets_2r = [
    (rotz,   0,  True),   # Rz(q1) - joint 0
    (transx, L1, False),  # Tx(L1) - constant
    (rotz,   1,  True),   # Rz(q2) - joint 1
    (transx, L2, False),  # Tx(L2) - constant
]

print("\n✓ Robot defined with DH and ETS")

In [ ]:
def ik_2r_analytical(x_target, y_target, L1, L2, elbow='up'):
    """
    Analytical IK for 2R planar robot.
    
    Parameters:
    - x_target, y_target: desired end-effector position
    - L1, L2: link lengths
    - elbow: 'up' or 'down'
    
    Returns:
    - q: [q1, q2] if solution exists, None otherwise
    """
    # Step 1: Distance to target
    D = sqrt(x_target**2 + y_target**2)
    
    # Step 2: Reachability check
    r_min = abs(L1 - L2)
    r_max = L1 + L2
    
    if D > r_max or D < r_min:
        print(f"  ✗ Target unreachable! D={D:.3f}, workspace=[{r_min:.3f}, {r_max:.3f}]")
        return None
    
    # Step 3: Solve for q2 (law of cosines)
    cos_q2 = (x_target**2 + y_target**2 - L1**2 - L2**2) / (2 * L1 * L2)
    
    # Numerical safety (round-off errors)
    cos_q2 = np.clip(cos_q2, -1, 1)
    
    if elbow == 'up':
        q2 = acos(cos_q2)  # Positive angle (elbow up)
    else:
        q2 = -acos(cos_q2)  # Negative angle (elbow down)
    
    # Step 4: Solve for q1
    alpha = atan2(y_target, x_target)
    beta = atan2(L2 * sin(q2), L1 + L2 * cos(q2))
    q1 = alpha - beta
    
    # Normalize to [-pi, pi]
    q1 = normalize_angle(q1)
    q2 = normalize_angle(q2)
    
    return np.array([q1, q2])


print("✓ Analytical IK function defined")

## Task 1.3: Testing & Validation (30 min)

In [ ]:
print("\n" + "="*70)
print("  2R PLANAR - ANALYTICAL IK TESTS")
print("="*70)

test_targets = [
    (0.5, 0.3, "Moderate reach"),
    (0.6, 0.2, "Near workspace boundary"),
    (0.0, 0.7, "Straight up (max reach)"),
    (0.2, 0.0, "Horizontal"),
]

solutions = []

for x_t, y_t, desc in test_targets:
    print(f"\n[Target: ({x_t:.2f}, {y_t:.2f}) - {desc}]")
    
    for elbow in ['up', 'down']:
        q = ik_2r_analytical(x_t, y_t, L1, L2, elbow=elbow)
        
        if q is not None:
            print(f"  {elbow.upper():4s}: q = [{np.rad2deg(q[0]):6.1f}°, {np.rad2deg(q[1]):6.1f}°]", end="")
            
            # Verify with FK using DH
            T_dh, _ = fk_dh(dh_2r, q)
            pos_dh = extract_position(T_dh)
            error_dh = np.linalg.norm([pos_dh[0] - x_t, pos_dh[1] - y_t])
            
            # Verify with FK using ETS
            T_ets, _ = fk_ets(ets_2r, q)
            pos_ets = extract_position(T_ets)
            error_ets = np.linalg.norm([pos_ets[0] - x_t, pos_ets[1] - y_t])
            
            print(f" → Error: DH={error_dh:.2e}, ETS={error_ets:.2e}")
            
            # Store first target solutions for visualization
            if x_t == test_targets[0][0] and y_t == test_targets[0][1]:
                solutions.append((q, f"Elbow {elbow.upper()}"))

print("\n" + "="*70)
print("\n✓ Both DH and ETS give identical results (modern approach validated!)")

In [ ]:
# Visualize both solutions for first target
if len(solutions) >= 2:
    x_target, y_target = test_targets[0][0], test_targets[0][1]
    compare_solutions(dh_2r, solutions, (x_target, y_target),
                     title="2R Planar - Elbow Up vs Elbow Down",
                     method='dh')
    
    # Also show with ETS
    print("\nShowing same solutions with ETS representation:")
    compare_solutions(ets_2r, solutions, (x_target, y_target),
                     title="2R Planar - ETS Representation (Modern)",
                     method='ets')

---

# Exercise 2: Workspace Analysis

**Time**: 50 minutes

## Task 2.1: Workspace Visualization (25 min)

In [ ]:
def plot_workspace_2r(L1, L2, n_samples=100):
    """
    Visualize 2R workspace and test IK at various points.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # Left plot: Workspace boundaries
    theta = np.linspace(0, 2*pi, 100)
    
    # Outer boundary (max reach)
    r_max = L1 + L2
    x_max = r_max * np.cos(theta)
    y_max = r_max * np.sin(theta)
    ax1.plot(x_max, y_max, 'b-', linewidth=2, label=f'Max reach = {r_max:.1f} m')
    ax1.fill(x_max, y_max, alpha=0.1, color='blue')
    
    # Inner boundary (min reach)
    r_min = abs(L1 - L2)
    if r_min > 0:
        x_min = r_min * np.cos(theta)
        y_min = r_min * np.sin(theta)
        ax1.plot(x_min, y_min, 'r-', linewidth=2, label=f'Min reach = {r_min:.1f} m')
        ax1.fill(x_min, y_min, alpha=0.1, color='white')
    
    ax1.plot(0, 0, 'ko', markersize=10, label='Base')
    ax1.set_xlabel('X (m)', fontsize=12)
    ax1.set_ylabel('Y (m)', fontsize=12)
    ax1.set_title('2R Planar Workspace', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    
    # Right plot: Sample IK solutions
    n_test = 20
    success_count = 0
    
    for i in range(n_test):
        # Random point in workspace
        angle = np.random.uniform(0, 2*pi)
        radius = np.random.uniform(r_min, r_max)
        x_test = radius * np.cos(angle)
        y_test = radius * np.sin(angle)
        
        # Try IK
        q = ik_2r_analytical(x_test, y_test, L1, L2, 'up')
        
        if q is not None:
            success_count += 1
            _, T_list = fk_dh(dh_2r, q)
            x_joints = [T[0,3] for T in T_list]
            y_joints = [T[1,3] for T in T_list]
            
            ax2.plot(x_joints, y_joints, 'o-', alpha=0.3, linewidth=1)
            ax2.plot(x_test, y_test, 'g.', markersize=6)
    
    ax2.plot(0, 0, 'ko', markersize=10, label='Base')
    ax2.set_xlabel('X (m)', fontsize=12)
    ax2.set_ylabel('Y (m)', fontsize=12)
    ax2.set_title(f'Sample Configurations ({success_count}/{n_test} successful)', 
                  fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.axis('equal')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nWorkspace analysis:")
    print(f"  Success rate: {success_count}/{n_test} = {100*success_count/n_test:.0f}%")
    print(f"  Workspace area: π({r_max:.2f}² - {r_min:.2f}²) = {pi*(r_max**2 - r_min**2):.3f} m²")


plot_workspace_2r(L1, L2)

## Task 2.2: Edge Cases & Singularities (25 min)

In [ ]:
print("\n" + "="*70)
print("  EDGE CASES & SINGULARITIES")
print("="*70)

edge_cases = [
    (L1+L2, 0.0, "Full extension (singularity)"),
    (0.0, L1+L2, "Full extension vertical"),
    (abs(L1-L2), 0.0, "Full retraction (singularity)"),
    (L1+L2+0.01, 0.0, "Beyond workspace"),
    (0.0, 0.0, "At base (singularity)"),
]

for x_t, y_t, desc in edge_cases:
    print(f"\n[{desc}] Target: ({x_t:.3f}, {y_t:.3f})")
    
    q = ik_2r_analytical(x_t, y_t, L1, L2, 'up')
    
    if q is not None:
        print(f"  ✓ Solution: q = [{np.rad2deg(q[0]):6.1f}°, {np.rad2deg(q[1]):6.1f}°]")
        
        # Check if at singularity (q2 ≈ 0 or ±π)
        if abs(q[1]) < 0.01 or abs(abs(q[1]) - pi) < 0.01:
            print(f"  ⚠ SINGULARITY: q2 ≈ {np.rad2deg(q[1]):.1f}° (extended/retracted)")
    else:
        print(f"  ✗ No solution")

print("\n" + "="*70)
print("\n**Observation**: Singularities occur at workspace boundaries (q2 = 0 or ±180°)")
print("At singularities, small motions in Cartesian space require large joint velocities!")

---

# Exercise 3: RTB Validation

**Time**: 50 minutes

## Task 3.1: Create RTB Robot (20 min)

In [ ]:
if RTB_AVAILABLE:
    print("\n" + "="*70)
    print("  RTB ROBOT CREATION")
    print("="*70)
    
    # Method 1: From DH parameters
    robot_dh = create_rtb_robot_from_dh(dh_2r, name="2R_DH")
    print(f"\n✓ Robot from DH:")
    print(robot_dh)
    
    # Method 2: From ETS (Modern approach!)
    robot_ets = create_rtb_robot_from_ets(ets_2r, name="2R_ETS")
    if robot_ets:
        print(f"\n✓ Robot from ETS (modern):")
        print(robot_ets)
    
    # Test FK consistency
    q_test = np.array([pi/4, pi/6])
    
    # Our implementations
    T_our_dh, _ = fk_dh(dh_2r, q_test)
    T_our_ets, _ = fk_ets(ets_2r, q_test)
    
    # RTB
    T_rtb = robot_dh.fkine(q_test)
    
    print(f"\n** FK Consistency Check **")
    print(f"Our DH:  {extract_position(T_our_dh)[:2]}")
    print(f"Our ETS: {extract_position(T_our_ets)[:2]}")
    print(f"RTB:     {T_rtb.t[:2]}")
    print(f"Match: {np.allclose(extract_position(T_our_dh)[:2], T_rtb.t[:2])}")
    
else:
    print("\nRTB not available - skipping")

## Task 3.2: Compare IK Methods (30 min)

In [ ]:
if RTB_AVAILABLE:
    print("\n" + "="*70)
    print("  IK METHOD COMPARISON: Analytical vs RTB Numerical")
    print("="*70)
    
    test_points = [
        (0.5, 0.3),
        (0.4, 0.4),
        (0.2, 0.6),
    ]
    
    for x_t, y_t in test_points:
        print(f"\n[Target: ({x_t}, {y_t})]")
        
        # Our analytical
        q_analytical = ik_2r_analytical(x_t, y_t, L1, L2, 'up')
        
        if q_analytical is not None:
            print(f"  Analytical: q = [{np.rad2deg(q_analytical[0]):6.1f}°, {np.rad2deg(q_analytical[1]):6.1f}°]")
            
            # RTB numerical IK (Levenberg-Marquardt)
            T_target = SE3(x_t, y_t, 0)
            sol_rtb = robot_dh.ikine_LM(T_target, mask=[1,1,0,0,0,0])  # Only XY
            
            if sol_rtb.success:
                q_rtb = sol_rtb.q
                print(f"  RTB (LM):   q = [{np.rad2deg(q_rtb[0]):6.1f}°, {np.rad2deg(q_rtb[1]):6.1f}°]")
                
                # Compare
                diff = np.rad2deg(q_rtb - q_analytical)
                print(f"  Difference:     [{diff[0]:6.1f}°, {diff[1]:6.1f}°]")
                
                if np.linalg.norm(diff) < 1.0:
                    print(f"  ✓ MATCH (same solution)")
                else:
                    print(f"  ⚠ Different solution (possibly elbow up/down)")
            else:
                print(f"  ✗ RTB failed to converge")
    
    print("\n" + "="*70)
    print("\n**Key insight**: RTB numerical methods find ONE solution,")
    print("while analytical gives us BOTH. Analytical is also FASTER!")
    
else:
    print("RTB not available")

---

# Exercise 4: Performance Analysis

**Time**: 40 minutes

## Task 4.1: Speed Comparison (20 min)

In [ ]:
import time

print("\n" + "="*70)
print("  PERFORMANCE BENCHMARK")
print("="*70)

n_trials = 1000
times_analytical = []
times_rtb = []

print(f"\nRunning {n_trials} IK computations...")

for i in range(n_trials):
    # Random target in workspace
    angle = np.random.uniform(0, 2*pi)
    radius = np.random.uniform(abs(L1-L2)+0.05, L1+L2-0.05)
    x_t = radius * np.cos(angle)
    y_t = radius * np.sin(angle)
    
    # Analytical
    start = time.time()
    q_analytical = ik_2r_analytical(x_t, y_t, L1, L2, 'up')
    times_analytical.append(time.time() - start)
    
    # RTB (if available)
    if RTB_AVAILABLE:
        T_target = SE3(x_t, y_t, 0)
        start = time.time()
        sol_rtb = robot_dh.ikine_LM(T_target, mask=[1,1,0,0,0,0])
        times_rtb.append(time.time() - start)

# Results
print(f"\nAnalytical IK:")
print(f"  Mean time: {np.mean(times_analytical)*1e6:.1f} μs")
print(f"  Std dev:   {np.std(times_analytical)*1e6:.1f} μs")

if RTB_AVAILABLE and len(times_rtb) > 0:
    print(f"\nRTB Numerical IK (Levenberg-Marquardt):")
    print(f"  Mean time: {np.mean(times_rtb)*1e3:.2f} ms")
    print(f"  Std dev:   {np.std(times_rtb)*1e3:.2f} ms")
    
    speedup = np.mean(times_rtb) / np.mean(times_analytical)
    print(f"\n** Analytical is {speedup:.0f}× FASTER than numerical! **")

# Visualization
if RTB_AVAILABLE and len(times_rtb) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(np.array(times_analytical)*1e6, bins=50, alpha=0.7, label='Analytical (μs)', color='blue')
    ax.hist(np.array(times_rtb)*1e3, bins=50, alpha=0.7, label='RTB Numerical (ms)', color='red')
    ax.set_xlabel('Computation Time', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title('IK Performance Comparison', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.show()

print("\n" + "="*70)

---

# Session 1 Summary

## What We Learned

✅ **IK Problem**: Converting desired position to joint angles

✅ **Analytical Solution**: Closed-form equations using geometry/trigonometry

✅ **Multiple Solutions**: 2R has 2 solutions (elbow up/down)

✅ **Workspace**: Annular region between min and max reach

✅ **Singularities**: At full extension/retraction (q2 = 0 or ±180°)

✅ **Modern Approach**: ETS representation (consistent with TP02)

✅ **Validation**: RTB confirms our results

✅ **Performance**: Analytical IK is ~1000× faster than numerical!

---

## Key Insights

**When analytical IK exists (like 2R, 3R, 6R with spherical wrist)**:
- Use it! Much faster and gives all solutions
- Critical for real-time control

**For general robots**:
- Numerical methods required (Session 2)
- Slower but work for any robot geometry

---

## Next Session

In Session 2, we'll explore:
- 3R planar robot (more complex analytical IK)
- 3R anthropomorphic (3D space, decoupling method)
- Numerical IK methods (Newton-Raphson, Damped Least Squares)
- Comparison of analytical vs numerical approaches

---

**Great work! You've mastered 2R analytical IK! 🎓**